## Prepare and read data

In [69]:
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import mean_squared_error

In [56]:
pd.set_option("max_columns", 300)
pd.set_option("max_rows", 500)

In [14]:
train_df = pd.read_csv("../input/train.csv")
test_df = pd.read_csv("../input/test.csv")

## Feature Engineering

In [4]:
def get_multi_cat_cols(train_df):
    tmp = train_df.iloc[:1000]
    multi_cols = []
    for c in train_df.columns:
        sep_num = tmp[c].astype(str).fillna("").str.contains(";").sum()
        if sep_num > 10:
            multi_cols.append(c)
    return multi_cols

In [15]:
original_cols = train_df.columns.tolist()

In [16]:
multi_cat_cols = get_multi_cat_cols(train_df)

In [17]:
for c in tqdm(multi_cat_cols):
    binarizer = MultiLabelBinarizer()
    train_multi_srs = train_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    test_multi_srs = test_df[c].map(lambda x: x.split(";") if x is not np.nan else [])
    train_arr = binarizer.fit_transform(train_multi_srs)
    test_arr = binarizer.transform(test_multi_srs)
    feat_cols = [f"{c}_{val}" for val in binarizer.classes_]
    train_feat_df = pd.DataFrame(train_arr, columns=feat_cols)
    test_feat_df = pd.DataFrame(test_arr, columns=feat_cols)
    train_df = pd.concat([train_df, train_feat_df], axis=1)
    test_df = pd.concat([test_df, test_feat_df], axis=1)

In [36]:
other_cols = [
    c for c in list(np.setdiff1d(original_cols, multi_cat_cols)) if c not in ["Salary", "No"]
]

In [37]:
for c in tqdm(other_cols):
    if train_df[c].dtype.name == "float":
        continue
    else:
        train_df[c], uniques = pd.factorize(train_df[c])
        test_df[c] = uniques.get_indexer(test_df[c])

In [38]:
train_df.shape

(33857, 444)

In [61]:
train_df.columns = ["".join (c if c.isalnum() else "_" for c in str(x)) for x in train_df.columns]

In [62]:
use_cols = [c for c in train_df.columns if c not in multi_cat_cols + ["Salary", "No"]]

## Training

In [63]:
folds = KFold(n_splits=5, random_state=2020, shuffle=True)

In [64]:
lgb_params = {
            'objective': 'regression',
            "metric": "rmse",
            "verbosity": -1,
            "boosting": "gbdt",
            'learning_rate': 0.05,
            'num_leaves': 128,
            'min_data_in_leaf': 30, 
            'max_depth': 6,
            "bagging_freq": 10,
            "bagging_fraction": 0.4,
            "lambda_l1": 0.5,
            "lambda_l2": 0.5,
            "feature_fraction": 0.7,
            "seed": 2020,
            "num_threads": -1,
            "max_bins": 30
}

In [68]:
importances = pd.DataFrame()
oof_preds = np.zeros(len(train_df))

for fold_i, (trn_idx, val_idx) in enumerate(folds.split(train_df, train_df["Salary"])):
    print(f"Fold {fold_i+1}")
    train_dataset = lgb.Dataset(
        train_df.loc[trn_idx, use_cols],
        label = train_df.loc[trn_idx, "Salary"]
    )
    valid_dataset = lgb.Dataset(
        train_df.loc[val_idx, use_cols],
        label = train_df.loc[val_idx, "Salary"]
    )
    model = lgb.train(
                lgb_params,
                train_dataset,
                3000,
                valid_sets = [train_dataset, valid_dataset],
                verbose_eval=100,
                early_stopping_rounds = 100,
                #feval = eval_f1,
                #callbacks = [log_callback],
    )
    imp_df = pd.DataFrame()
    imp_df['feature'] = use_cols
    imp_df['gain'] = model.feature_importance(importance_type="gain")
    importances = pd.concat([importances, imp_df], axis=0, sort=False)
    
    oof_preds[val_idx] = model.predict(train_df.loc[val_idx, use_cols])

Fold 1
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 21446.3	valid_1's rmse: 23183
[200]	training's rmse: 19556.2	valid_1's rmse: 22455
[300]	training's rmse: 18383.6	valid_1's rmse: 22180.9
[400]	training's rmse: 17463.2	valid_1's rmse: 22046.7
[500]	training's rmse: 16611.5	valid_1's rmse: 22014.9
[600]	training's rmse: 15793.5	valid_1's rmse: 21963.7
[700]	training's rmse: 15149.1	valid_1's rmse: 21916.3
[800]	training's rmse: 14457.7	valid_1's rmse: 21849.3
Early stopping, best iteration is:
[782]	training's rmse: 14567.8	valid_1's rmse: 21840
Fold 2
Training until validation scores don't improve for 100 rounds
[100]	training's rmse: 21569.3	valid_1's rmse: 23024.6
[200]	training's rmse: 19757.7	valid_1's rmse: 22196.7
[300]	training's rmse: 18499.2	valid_1's rmse: 21905
[400]	training's rmse: 17467.2	valid_1's rmse: 21746
[500]	training's rmse: 16640.8	valid_1's rmse: 21699.8
[600]	training's rmse: 15906.6	valid_1's rmse: 21655.9
[700]	traini

In [71]:
oof_score = np.sqrt(mean_squared_error(train_df["Salary"], oof_preds))
oof_score

21731.44156864013

In [73]:
importances.groupby("feature")["gain"].mean().sort_values(ascending=False).reset_index()

,feature,gain
0,MilitaryUS,4.854589e+13
1,YearsCodingProf,2.278303e+13
2,Country,1.849505e+13
3,SalaryType,1.340749e+13
4,CurrencySymbol,8.912159e+12
5,YearsCoding,8.715019e+12
6,Employment,6.041925e+12
7,DevType_Student,5.713235e+12
8,Age,5.073843e+12
9,Currency,4.897888e+12
